# 1. 라이브러리 선언하기

In [105]:
import pandas as pd
import numpy as np

# 2. 데이터 불러오기

In [106]:
selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

### 3-1. 데이터타입 조회

In [107]:
selloutTypeData = selloutData.astype({
    "REGIONID":str,
    "PRODUCT":str,
    "YEARWEEK":str,
    "QTY":float})

In [108]:
selloutTypeData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object

### 3-2. 불량데이터 처리

In [109]:
selloutTypeData["QTY_NEW"]= np.where(selloutTypeData.QTY<0,0,selloutTypeData.QTY)

In [110]:
selloutTypeData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0
...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0
124654,A10,PRODUCT56,201630,275.0,275.0
124655,A10,PRODUCT61,201630,0.0,0.0
124656,A10,PRODUCT12,201630,15021.0,15021.0


### 3-3. 데이터 통합

In [111]:
## YEARWEEK에서 나눠야할 연도, 주차 길이 정의
yearAndWeek = 4

## YearAndWeek를 기준으로 데이터 나누기
selloutTypeData["YEAR"]=selloutTypeData.YEARWEEK.str[:yearAndWeek]
selloutTypeData["WEEK"]=selloutTypeData.YEARWEEK.str[yearAndWeek:]

## 구하고자 하는 Week 변수 지정
yearlength = 52

## yearlength를 기준으로 데이터 찾기
findSelloutData = selloutTypeData.loc[selloutTypeData.WEEK.astype(int)<=yearlength]

## 인덱스 재설정
refinedSelloutData=findSelloutData.reset_index(drop=True)

## 검산식
# refinedSelloutData.loc[refinedSelloutData.WEEK.astype(int)>yearlength]

refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
123859,A10,PRODUCT60,201630,824.0,824.0,2016,30
123860,A10,PRODUCT56,201630,275.0,275.0,2016,30
123861,A10,PRODUCT61,201630,0.0,0.0,2016,30
123862,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


<!-- ## 3. 불량 데이터 처리

selloutTypeData["QTY_NEW"] = np.where(selloutTypeData.QTY<0,0,selloutTypeData.QTY)
selloutTypeData -->

In [112]:
refinedSelloutData.columns

Index(['REGIONID', 'PRODUCT', 'YEARWEEK', 'QTY', 'QTY_NEW', 'YEAR', 'WEEK'], dtype='object')

### 4-1 지역, 상품, 연주차 컬럼순으로 오름차순 정렬하기

In [113]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

sortedData = refinedSelloutData.sort_values(by=sortKey,inplace=False)

In [114]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
298,A00,PRODUCT34,201401,661.0,661.0,2014,01
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03
205,A00,PRODUCT34,201404,532.0,532.0,2014,04
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05


### 4-2 판매량(qty_new)의 평균 연산

In [115]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

groupData = sortedData.groupby(by=groupKey,as_index=False)["QTY_NEW"].agg(["mean"])

# 컬럼명 변경(mean => QTY_MEAN)
groupData.rename(columns={"mean":"QTY_MEAN"},inplace=True,)

# 인덱스 설정
groupData = groupData.reset_index()

### 4-3 데이터 조인

In [116]:
refinedSelloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02


In [117]:
groupData.head()

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538


In [118]:
joinKey = ["REGIONID","PRODUCT","YEAR"]
mergedData = pd.merge(refinedSelloutData,groupData, on=joinKey)

In [119]:
sortKey = ["REGIONID","PRODUCT","YEAR","WEEK"]

mergedData = mergedData.sort_values(by=sortKey,inplace=False)

In [120]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN
10661,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538
10662,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538
10663,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538
10660,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538
10665,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538
...,...,...,...,...,...,...,...,...
96903,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846
96893,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846
96904,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846
96898,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846


### 4-4 계절성지수

In [121]:
mergedData["SEASONALITY"]=mergedData["QTY_NEW"]/mergedData["QTY_MEAN"]

In [122]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
10661,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
10662,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
10663,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
10660,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
10665,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538,1.869826
...,...,...,...,...,...,...,...,...,...
96903,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846,0.858356
96893,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846,1.051445
96904,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
96898,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815


### 4-5 계절성 지수의 평균값

In [123]:
groupKeys = ["REGIONID","PRODUCT","WEEK"]

In [124]:
finalResult = mergedData.groupby(groupKeys)["SEASONALITY"].agg(["mean"])

In [125]:
finalResult.reset_index()

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192
